
# Importing

In [4]:
# colab setting
!pip install GraphRicciCurvature
!pip install scikit-learn

# import sys
# sys.path.append("../")


import networkx as nx
import numpy as np
import math
import random

# matplotlib setting
%matplotlib inline
import matplotlib.pyplot as plt

# to print logs in jupyter notebook
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

# load GraphRicciCuravture package
from GraphRicciCurvature.OllivierRicci import OllivierRicci
from GraphRicciCurvature.FormanRicci import FormanRicci


# load python-louvain for modularity computation
import community.community_louvain as community_louvain

# for ARI computation
from sklearn import preprocessing, metrics

In [2]:
import GraphRicciCurvature
print(GraphRicciCurvature.__version__)

0.5.3.1


# Helper Functions

In [8]:
def draw_graph(G, clustering_label="club"):
    """
    A helper function to draw a nx graph with community.
    """
    complex_list = nx.get_node_attributes(G, clustering_label)

    le = preprocessing.LabelEncoder()
    node_color = le.fit_transform(list(complex_list.values()))


    nx.draw_spring(G, nodelist=G.nodes(),
                   node_color=node_color,
                   cmap=plt.cm.rainbow,
                   alpha=0.8)



def show_results(G, curvature="ricciCurvature"):
    """
    A helper function to show summary of curvature in a graph

    # parameters
     - G : networkx graph
    # returns
     - plot of summaries
    """
    # Print the first five results
    print("Graph, first 5 edges: ")
    for n1,n2 in list(G.edges())[:5]:
        print("Ricci curvature of edge (%s,%s) is %f" % (n1 ,n2, G[n1][n2][curvature]))

    # Plot the histogram of Ricci curvatures
    plt.subplot(2, 1, 1)
    ricci_curvtures = nx.get_edge_attributes(G, curvature).values()
    plt.hist(ricci_curvtures,bins=20)
    plt.xlabel('Ricci curvature')
    plt.title("Histogram of Ricci Curvatures")

    # Plot the histogram of edge weights
    plt.subplot(2, 1, 2)
    weights = nx.get_edge_attributes(G, "weight").values()
    plt.hist(weights,bins=20)
    plt.xlabel('Edge weight')
    plt.title("Histogram of Edge weights")

    plt.tight_layout()


def ARI(G, clustering, clustering_label="club"):
    """
    Computer the Adjust Rand Index (clustering accuracy) of "clustering" with "clustering_label" as ground truth.

    Parameters
    ----------
    G : NetworkX graph
        A given NetworkX graph with node attribute "clustering_label" as ground truth.
    clustering : dict or list or list of set
        Predicted community clustering.
    clustering_label : str
        Node attribute name for ground truth.

    Returns
    -------
    ari : float
        Adjust Rand Index for predicted community.
    """

    complex_list = nx.get_node_attributes(G, clustering_label)

    le = preprocessing.LabelEncoder()
    y_true = le.fit_transform(list(complex_list.values()))

    if isinstance(clustering, dict):
        # python-louvain partition format
        y_pred = np.array([clustering[v] for v in complex_list.keys()])
    elif isinstance(clustering[0], set):
        # networkx partition format
        predict_dict = {c: idx for idx, comp in enumerate(clustering) for c in comp}
        y_pred = np.array([predict_dict[v] for v in complex_list.keys()])
    elif isinstance(clustering, list):
        # sklearn partition format
        y_pred = clustering
    else:
        return -1

    return metrics.adjusted_rand_score(y_true, y_pred)


def my_surgery(G_origin: nx.Graph(), weight="weight", cut=0):
    """A simple surgery function that remove the edges with weight above a threshold

    Parameters
    ----------
    G_origin : NetworkX graph
        A graph with ``weight`` as Ricci flow metric to cut.
    weight: str
        The edge weight used as Ricci flow metric. (Default value = "weight")
    cut: float
        Manually assigned cutoff point.

    Returns
    -------
    G : NetworkX graph
        A graph after surgery.
    """
    G = G_origin.copy()
    w = nx.get_edge_attributes(G, weight)

    assert cut >= 0, "Cut value should be greater than 0."
    if not cut:
        cut = (max(w.values()) - 1.0) * 0.6 + 1.0  # Guess a cut point as default

    to_cut = []
    for n1, n2 in G.edges():
        if G[n1][n2][weight] > cut:
            to_cut.append((n1, n2))
    print("*************** Surgery time ****************")
    print("* Cut %d edges." % len(to_cut))
    G.remove_edges_from(to_cut)
    print("* Number of nodes now: %d" % G.number_of_nodes())
    print("* Number of edges now: %d" % G.number_of_edges())
    cc = list(nx.connected_components(G))
    print("* Modularity now: %f " % nx.algorithms.community.quality.modularity(G, cc))
    print("* ARI now: %f " % ARI(G, cc))
    print("*********************************************")

    return G

# Missing Edges Helpers

In [9]:
def block_model_with_edge_removal(block_sizes, prob_matrix, p = 0, method="uniform"):
    """
    Generates a block model graph and removes edges according to the specified method:
    uniformly, by degree, or based on inter-community probabilities.

    Parameters:
    - block_sizes : array-like, the size of each community.
    - prob_matrix : array-like, matrix of probabilities for edge creation between communities.
    - p : float, array-like, or matrix, specifying the probability or proportions of edges to remove.
      Interpreted based on the 'method' parameter.
    - method : str, method for edge removal ("uniform", "degree", or "weighted").

    Returns:
    - G : networkx graph, a graph with edges removed according to the specified method.
    """

    # Generate a stochastic block model graph
    G = nx.generators.community.stochastic_block_model(block_sizes, prob_matrix)

    # Assign 'block' attribute to each node based on its community
    club_attribute = {node: G.nodes[node]['block'] for node in G.nodes}
    nx.set_node_attributes(G, club_attribute, name='club')

    if method == "uniform" or method == "degree":
        edges_to_consider = list(G.edges(data=True))
    else:
        edges_to_consider = G.edges(data='block')

    if method == "uniform":
        for edge in edges_to_consider:
            if random.random() < p:
                G.remove_edge(edge[0], edge[1])

    elif method == "degree":
        for node in list(G.nodes()):
            adjacent_edges = list(G.edges(node))
            num_to_remove = int(len(adjacent_edges) * p)
            edges_to_remove = random.sample(adjacent_edges, num_to_remove)
            G.remove_edges_from(edges_to_remove)

    elif method == "weighted":
        if not isinstance(p, (list, np.ndarray)) or np.shape(p) != (len(block_sizes), len(block_sizes)):
            raise ValueError("For 'weighted' method, p must be a matrix matching the number of communities.")

        for edge in edges_to_consider:
            source_block = G.nodes[edge[0]]['block']
            target_block = G.nodes[edge[1]]['block']
            removal_prob = p[source_block][target_block]

            if random.random() < removal_prob:
                G.remove_edge(edge[0], edge[1])

    else:
        raise ValueError("Invalid method specified. Choose 'uniform', 'degree', or 'weighted'.")

    return G
